In [1]:
# Day_03_01_auto_encoder.py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def model_1(ph_x):
    enc_w = tf.get_variable('enc_w1', shape=[784, 256], initializer=tf.initializers.glorot_normal())
    enc_b = tf.Variable(tf.zeros([256]))

    enc = tf.matmul(ph_x, enc_w) + enc_b
    relu = tf.nn.relu(enc)

    dec_w = tf.get_variable('dec_w1', shape=[256, 784], initializer=tf.initializers.glorot_normal())
    dec_b = tf.Variable(tf.zeros([784]))

    dec = tf.matmul(relu, dec_w) + dec_b
    # 0~1을 벗어나면 noise가 나와서....
    # return tf.nn.relu(dec)
    return tf.nn.sigmoid(dec)

# 문제
## 1번 함수를 tf.layers로 수정하세요

In [3]:
def model_2(ph_x):

    output = tf.layers.dense(ph_x, 256, activation=tf.nn.relu)
    return tf.layers.dense(output, 784, activation=tf.nn.sigmoid)

In [4]:
def auto_encoder():
    mnist = input_data.read_data_sets('mnist')

    # ph_x = tf.placeholder(tf.float32)
    # tf.layer를 사용할때는 shape을 지정 해준다.
    # -1은 명확하게 존잰할때, None은 모를때 사용
    ph_x = tf.placeholder(tf.float32, shape=[None, 784])

    output = model_2(ph_x)

    # loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z, labels=ph_y)
    loss = tf.reduce_mean((ph_x - output) ** 2)

    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    epochs = 10
    batch_size = 100   # 출구를 몇개씩 조사할건지
    n_iters = mnist.train.num_examples // batch_size # 550

    # epcochs를 1번 돌고 념
    for i in range(epochs):
        c = 0
        for j in range(n_iters):
            xx, _ = mnist.train.next_batch(batch_size)

            sess.run(train, feed_dict={ph_x: xx})
            c += sess.run(loss, {ph_x: xx})

        print(i, c/ n_iters)

    # ---------------------------------------------------------------- #
    sample_count = 10
    samples = sess.run(output, {ph_x: mnist.test.images[:sample_count]})

    _, ax = plt.subplots(2, sample_count, figsize=(sample_count, 2))

    for i in range(sample_count):
        ax[0, i].set_axis_off()
        ax[1, i].set_axis_off()

        ax[0][i].imshow(np.reshape(mnist.test.images[i], [28, 28]), cmap='gray')
        ax[1][i].imshow(np.reshape(samples[i], [28, 28]), cmap='gray')

        # ax[0][i].imshow(np.reshape(mnist.test.images[i], [28, 28]))
        # ax[1][i].imshow(np.reshape(samples[i], [28, 28]))

    plt.tight_layout()
    plt.show()

    sess.close()

In [ ]:
auto_encoder()